DISTANZA DI MAHALANOBIS (supponendo le componenti non correlate)
Simile alla distanza euclidea multidimensionale ma tiene conto della varianza di ogni feature. La normalizzazione componente per componente fa si che una deviazione su una componente con varianza elevata ha peso minore rispetto ad una deviazione di uguale valore assoluto che avviene su una componente meno variabile.
Dobbiamo decidere una soglia per rilevare il DRIFT.
1,50 Allarme, 2,00 Drift

In [39]:
import pandas as pd
import numpy as np
import csv
import skmultiflow as sk


size=50
dataset=pd.read_csv('SRdrift.csv' ,usecols=["A","B","C"])
data_stream= sk.data.DataStream(dataset ,y=None, n_targets=0)
data_stream.print_df()
data_stream.prepare_for_use()
 
data=data_stream.next_sample(batch_size=size)[0]
old_w=pd.DataFrame(data, columns=['A','B','C'])
print(f"Old:{old_w}")


[[-258.93763863 -654.19491326 -570.7438663 ]
 [-258.70523191 -655.15165221 -570.92558994]
 [-259.4375307  -653.76023233 -570.98461852]
 ...
 [-258.82138476 -654.07704964 -571.45474237]
 [-259.22645187 -654.94091298 -570.51667721]
 [-259.7429683  -654.39650835 -572.13457074]]
[]
Old:              A            B           C
0   -258.937639  -654.194913 -570.743866
1   -258.705232  -655.151652 -570.925590
2   -259.437531  -653.760232 -570.984619
3   -259.121990  -654.550492 -571.737737
4   -259.557844  -653.905345 -571.012381
5  -1272.811696 -1073.448771 -674.846233
6  -1273.050592 -1072.934483 -676.748875
7   -259.400233  -654.490069 -570.997042
8   -259.440612  -654.584772 -571.592359
9   -259.119634  -654.582419 -571.713258
10  -259.892008  -653.949732 -570.899341
11  -259.785050  -654.483813 -571.143544
12  -259.691276  -653.730841 -571.191166
13  -259.871669  -653.819641 -570.901085
14 -1274.109718 -1073.430454 -675.318520
15  -259.207896  -654.524699 -571.494482
16  -259.174357  -65

c:\Users\Salvatore Drago\Desktop\progetto_lab\lib\site-packages\skmultiflow\data\base_stream.py:194: FutureWarning: 'prepare_for_use' has been deprecated in v0.5.0 and will be removed in v0.7.0.
New instances of the Stream class are now ready to use after instantiation.
  category=FutureWarning)


In [109]:
data=data_stream.next_sample(batch_size=size)[0]
new_w=pd.DataFrame(data, columns=['A','B','C'])
print(f"New:{new_w}")

New:              A            B           C
0     99.100979    99.959137  -29.428166
1     99.064069    99.795323  -29.276809
2     99.090885    99.713553  -29.251473
3   -479.509590   273.963372  466.902075
4     99.056655    99.967181  -29.459309
5   -478.759639   272.982045  467.522133
6     98.773541   100.003931  -29.395711
7     98.910739    99.915606  -29.347114
8     98.898427    99.884179  -29.283910
9   -478.510043   273.631735  466.003833
10    98.788657   100.054006  -29.511122
11    99.130076    99.930913  -29.332330
12    99.100991   100.063631  -29.297855
13    98.956235    99.997321  -29.377874
14  -478.853027   274.063429  467.352339
15    98.978577    99.946317  -29.547222
16    98.777656   100.005116  -29.267601
17    99.074019    99.813385  -29.383604
18  -478.601959   273.545122  466.457984
19    98.913259    99.941702  -29.153433
20    99.031926    99.716178  -29.199260
21    98.840797   100.000175  -29.223330
22 -1276.510716 -1074.186947 -674.764818
23    99.020

In [110]:
df=pd.merge(old_w, new_w, how='outer')
print(df)





             A           B           C
0  -258.937639 -654.194913 -570.743866
1  -258.705232 -655.151652 -570.925590
2  -259.437531 -653.760232 -570.984619
3  -259.121990 -654.550492 -571.737737
4  -259.557844 -653.905345 -571.012381
..         ...         ...         ...
95   98.766563   99.875604  -29.183929
96   98.995751   99.785678  -29.427856
97   98.985664   99.782940  -29.302660
98   98.897559   99.992909  -29.277720
99   98.984779   99.919394  -29.467548

[100 rows x 3 columns]


In [111]:
var= df.var(axis=0,ddof=0)
print(var)


A    112804.088012
B    180417.093750
C    108197.899257
dtype: float64


In [112]:
def mahalanobis_dist(old,new,var):
    old_np=old.to_numpy()
    new_np=new.to_numpy()
    var_np=var.to_numpy()
    dist_tot=0
    n_elem=0
    for j in range (0, len(old_np)):
        dist=0
        for i in range (0, len(old_np[0])):
            #print(f"VAR:{var_np[i]}")
            dist=dist+((1/var_np[i])*((old_np[j][i]-new_np[j][i])**2))
        dist_tot=dist_tot+(dist**(1/2))
        #print(f"Dist:{dist}, Dist_tot:{dist_tot}")
        n_elem+=1
    dist_tot_mean=dist_tot/n_elem
    print(f"Dist_tot:{dist_tot}, n_elem:{n_elem}, dist_tot_mean:{dist_tot/n_elem}    {dist_tot_mean}")

In [113]:
mahalanobis_dist(old_w,new_w,var)

Dist_tot:148.9934642159873, n_elem:50, dist_tot_mean:2.979869284319746    2.979869284319746
